For all k-wta network, we just adjust the ResNet18, if you want to use, please add code in follow block to the SparseResNet Class __init__ funciton in kWTA/resnet.py and replace the relu with sp in forwar function. Then use the sparseresnet.

In [ ]:
self.sp = models.sparse_func_dict[sparse_func](sparsities[0])

For speed up but failed k-wta, please add code follow to kWTA/models.py and add a name in dict to call.

In [ ]:
class Sparsify2D_spvol(SparsifyBase):
    def __init__(self, sparse_ratio=0.5, sample_rate=0.1):
        super(Sparsify2D_spvol, self).__init__()
        self.sr = sparse_ratio
        self.relu = nn.ReLU()
        self.sr2 = sample_rate
    def forward(self, x):
        size = x.shape[1]*x.shape[2]*x.shape[3]
        temp = torch.randint(size,(1,int(self.sr2*size)),device=x.device)
        tmpx = x.view(x.shape[0], -1)
        realt = tmpx[[0],temp[0]]
        k = int(0.2*realt.shape[0])
        topval = realt.topk(k, dim=0)[0][-1]
        return self.relu(x-topval)

Sample: Prepare the data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import json
import time
import sys
import copy

from kWTA import models
from kWTA import activation
from kWTA import attack
from kWTA import training
from kWTA import utilities
from kWTA import densenet
from kWTA import resnet
from kWTA import wideresnet
norm_mean = 0
norm_var = 1
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((norm_mean,norm_mean,norm_mean), (norm_var, norm_var, norm_var)),
])
cifar_train = datasets.CIFAR10("./data", train=True, download=True, transform=transform_train)
cifar_test = datasets.CIFAR10("./data", train=False, download=True, transform=transform_test)
train_loader = DataLoader(cifar_train, batch_size = 256, shuffle=True)
test_loader = DataLoader(cifar_test, batch_size = 100, shuffle=True)

Feed the data to model

In [ ]:
device = torch.device('cuda:0')
model = resnet.SparseResNet18(sparsities=[0.1,0.1,0.1,0.1], sparse_func='vol').to(device)
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
eps = 0.031
epochs = 80
hist = torch.zeros((6,epochs))
for ep in range(epochs):
    if ep == 50:
        for param_group in opt.param_groups:
                param_group['lr'] = 0.01
    train_err, train_loss = training.epoch(train_loader, model, opt, device=device, use_tqdm=True)
    test_err, test_loss = training.epoch(test_loader, model, device=device, use_tqdm=True)
    adv_err, adv_loss = training.epoch_adversarial(test_loader,
        model, attack=attack.pgd_linf_untargeted, device=device, num_iter=20, 
        use_tqdm=True, epsilon=eps, randomize=True, alpha=0.003, n_test=1000)
    hist[0][ep]=train_loss
    hist[1][ep]=train_err
    hist[2][ep]=test_loss
    hist[3][ep]=test_err
    hist[4][ep]=adv_loss
    hist[5][ep]=adv_err
    print('epoch', ep, 'train err', train_err, 'test err', test_err, 'adv_err', adv_err, "train_loss", train_loss)
    torch.save(model.state_dict(), 'models/spresnet18_01_cifar_{}epoch.pth'.format(ep))

Then use them to plot the curves

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].plot(hist[0],label = 'train_loss',linestyle=':')
ax[0].plot(hist[2], label = 'test_loss',linestyle='--')
ax[0].plot(hist[4], label = 'adv_loss')
ax[0].legend(loc = 'mid right')
ax[0].grid(True)
ax[0].set_title("All k-WTA 0.2", fontsize=14)
ax[1].plot(1-hist[1],label = 'train_acc',linestyle=':')
ax[1].plot(1-hist[3], label = 'test_acc',linestyle='--')
ax[1].plot(1-hist[5], label = 'adv_acc',linestyle='-.')
ax[1].legend(loc = 'mid right')
ax[1].grid(True)
ax[1].set_title("CIFAR10 80epochs", fontsize=14)

Then add code in follow block to the kWTA/attack.py, which can provide the ability to attack via C&W and MIM.

In [ ]:
from advertorch.attacks import CarliniWagnerL2Attack
def CW(model,X,y,num_class=10,num_iter=10):
    adversary = CarliniWagnerL2Attack(model, loss_fn=nn.CrossEntropyLoss(reduction="sum"),num_classes=num_class,confidence=0, targeted=False, learning_rate=0.01, binary_search_steps=5, max_iterations=20, abort_early=True, initial_const=0.001, clip_min=0.0, clip_max=1.0)
    adv_untargeted = adversary.perturb(X, y)-X
    return adv_untargeted


from advertorch.attacks import MomentumIterativeAttack
def MIM(model,X,y,num_iter=10):
    adversary = MomentumIterativeAttack(model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.3, nb_iter=10, decay_factor=1.0, eps_iter=0.003, clip_min=0.0, clip_max=1.0)
    adv_untargeted = adversary.perturb(X, y)-X
    return adv_untargeted